<a href="https://colab.research.google.com/github/rishurnjan/outfit_recommendation/blob/main/outfit_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
 98% 1.31G/1.33G [00:22<00:00, 93.5MB/s]
100% 1.33G/1.33G [00:22<00:00, 64.4MB/s]


In [ ]:
!kaggle datasets download -d zalando-research/fashionmnist --unzip

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
 99% 68.0M/68.8M [00:00<00:00, 174MB/s]
100% 68.8M/68.8M [00:00<00:00, 143MB/s]


In [ ]:
import pandas as pd
import numpy as np

celeba_df=pd.read_csv("/content/list_attr_celeba.csv")
mnist_train_df=pd.read_csv("/content/fashion-mnist_train.csv")
mnist_test_df=pd.read_csv("/content/fashion-mnist_test.csv")
# Extracting and reading of datasets is completed

In [ ]:
celeba_df.head()

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1


In [ ]:
mnist_train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0 T-shirt/top 1 Trouser 2 Pullover 3 Dress 4 Coat 5 Sandal 6 Shirt 7 Sneaker 8 Bag 9 Ankle boot

Formal= Trouser, Shirt, Ankle boot,Dress, Coat
Informal/Casual= Tshirt/top, Pullover, Sandal,Sneaker, Bag

In [ ]:
celeba_df.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [ ]:
features_informal=['Wearing_Hat','Wavy_Hair','Goatee','Chubby','Blurry',
                   'Bags_Under_Eyes','Young']

features_formal=['Wearing_Necktie','Mustache','Bald']

'''
These are harcode informal and formal features.
Introducing two categories here as formal and informal under Style feature.
Rest of the features can be alloted randomly'''

'\nThese are harcode informal and formal features.\nIntroducing two categories here as formal and informal under Style feature. \nRest of the features can be alloted randomly'

In [ ]:
leftout_features = set(celeba_df.columns) - set(features_informal) - set(features_formal)
leftout_features = list(leftout_features)
leftout_features.remove('image_id')
leftout_features

['Pointy_Nose',
 'Smiling',
 'Gray_Hair',
 'Blond_Hair',
 'No_Beard',
 'Wearing_Necklace',
 'Straight_Hair',
 'Sideburns',
 'Oval_Face',
 'Attractive',
 'Bangs',
 'Male',
 '5_o_Clock_Shadow',
 'Wearing_Lipstick',
 'Brown_Hair',
 'Heavy_Makeup',
 'Eyeglasses',
 'Big_Nose',
 'Pale_Skin',
 'Big_Lips',
 'Arched_Eyebrows',
 'High_Cheekbones',
 'Black_Hair',
 'Narrow_Eyes',
 'Receding_Hairline',
 'Mouth_Slightly_Open',
 'Bushy_Eyebrows',
 'Wearing_Earrings',
 'Double_Chin',
 'Rosy_Cheeks']

In [ ]:
import random
random.shuffle(leftout_features)
mid=len(leftout_features)//2
informal_features=leftout_features[:mid]
formal_features=leftout_features[mid:]

In [ ]:
informal_features

['Double_Chin',
 'Pale_Skin',
 'Straight_Hair',
 'Wearing_Necklace',
 'Mouth_Slightly_Open',
 'Smiling',
 'Receding_Hairline',
 'Male',
 'Big_Lips',
 'Oval_Face',
 'Gray_Hair',
 'Pointy_Nose',
 'Blond_Hair',
 'Sideburns',
 'Black_Hair']

In [ ]:
formal_features

['Bangs',
 'Wearing_Lipstick',
 '5_o_Clock_Shadow',
 'No_Beard',
 'Big_Nose',
 'Heavy_Makeup',
 'Narrow_Eyes',
 'Arched_Eyebrows',
 'Eyeglasses',
 'Attractive',
 'Rosy_Cheeks',
 'Brown_Hair',
 'Bushy_Eyebrows',
 'High_Cheekbones',
 'Wearing_Earrings']

In [ ]:
celeba_df.shape

(202599, 41)

In [ ]:
# Spliting the celeba dataset for training and testing
from sklearn.model_selection import train_test_split
celeba_train,celeba_test=train_test_split(celeba_df,test_size=0.2,random_state=42)

In [ ]:
celeba_train.reset_index(drop=True,inplace=True)
celeba_test.reset_index(drop=True,inplace=True)

In [ ]:
#Taking the first 100 image_ids and alloting their Style category
celeba_train_sub1=celeba_train.loc[:100]
celeba_train_sub2=celeba_train.loc[100:]

In [ ]:
celeba_train_sub2.shape

(161979, 41)

In [ ]:
for row in range(0,101):
  formal_score= sum(celeba_train_sub1.loc[row,feature]==1 for feature in formal_features)
  informal_score= sum(celeba_train_sub1.loc[row,feature]==1 for feature in informal_features)
  if formal_score>informal_score:
    celeba_train_sub1.loc[row,'Style']=1
  else:
    celeba_train_sub1.loc[row,'Style']=-1

<ipython-input-19-4762af6f967d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeba_train_sub1.loc[row,'Style']=-1


In [ ]:
# Remaining image_ids Style category allotments happens through logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
Y=celeba_train_sub1['Style']
X=celeba_train_sub1.drop(['Style','image_id'],axis=1)

model=LogisticRegression()
model.fit(X,Y)
y_pred=model.predict(celeba_train_sub2.drop(['image_id'],axis=1))



Style category
1-> formal
-1->informal

In [ ]:
'''Style Category
1-> formal
-1->informal'''
celeba_train_sub2['Style']=y_pred
celeba_train_sub2.head()

<ipython-input-21-e023feeba30a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  celeba_train_sub2['Style']=y_pred


,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,Style
100,156395.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,1,-1.0
101,105918.jpg,1,-1,1,-1,-1,1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1.0
102,107075.jpg,-1,-1,-1,-1,-1,1,1,-1,-1,...,-1,-1,-1,1,-1,1,-1,-1,1,1.0
103,118562.jpg,-1,-1,1,1,-1,-1,-1,1,-1,...,1,-1,1,1,-1,1,-1,-1,1,1.0
104,086446.jpg,1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1.0


In [ ]:
#Joinning the celeba train datasets again into one whole.
celeba_train = pd.concat([celeba_train_sub1,celeba_train_sub2],axis=0)

In [ ]:
# Again training the model with all image_ids of celeba_train dataset. This will help us train the model more effectively for the test part.
Y=celeba_train['Style']
X=celeba_train.drop(['Style','image_id'],axis=1)

X_train,X_valid,y_train,y_valid=train_test_split(X,Y,test_size=0.2,random_state=42)

model.fit(X_train,y_train)
y_pred=model.predict(X_valid)
print(accuracy_score(y_valid,y_pred))

0.9989511352418559


In [ ]:
celeba_test['Style']=model.predict(celeba_test.drop(['image_id'],axis=1))


In [ ]:
#Assigning the outfit recommendations based on style
output_df=celeba_test[['image_id','Style']]
for i in range(output_df.shape[0]):
  if output_df.loc[i,'Style']==1:
    output_df.loc[i,'Style']='formal'
    output_df.loc[i,'Outfits_Recommendation']="Trouser, Shirt, Ankle boot,Dress,Coat"
  else:
    output_df.loc[i,'Style']='informal'
    output_df.loc[i,'Outfits_Recommendation']="Tshirt/top, Pullover, Sandal,Sneaker,Bag"

<ipython-input-25-25a0b03016d9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df.loc[i,'Outfits_Recommendation']="Tshirt/top, Pullover, Sandal,Sneaker,Bag"


In [ ]:
#The Required output
output_df.head()

,image_id,Style,Outfits_Recommendation
0,093242.jpg,informal,"Tshirt/top, Pullover, Sandal,Sneaker,Bag"
1,121414.jpg,formal,"Trouser, Shirt, Ankle boot,Dress,Coat"
2,010625.jpg,formal,"Trouser, Shirt, Ankle boot,Dress,Coat"
3,082524.jpg,formal,"Trouser, Shirt, Ankle boot,Dress,Coat"
4,073193.jpg,informal,"Tshirt/top, Pullover, Sandal,Sneaker,Bag"


In [ ]:
output_df.to_csv('output.csv',index=False)